In [1]:
import tensorflow as tf
from keras.preprocessing import image
import numpy as np
import keras
from keras import Model
import time
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import Conv3D

fileName = "Trump.jpg"



# Global MobileNets model
mobile = keras.applications.mobilenet.MobileNet(weights="imagenet")
model = Model(mobile.input, mobile.layers[7].output)


Using TensorFlow backend.


In [ ]:
def write_to_file_input(fileName, X):
    fhandler = open(fileName, "w")
    for i in range(3):
        for j in range(224):
            for k in range(224):
                fhandler.write(str(X[j][k][i]) + "\n")
    fhandler.close()

In [ ]:
def create_Input_Files():
    
    img_width = 224
    img_height = 224

    img = image.load_img(fileName, target_size=(img_width, img_height))
    X = image.img_to_array(img)

    # Normalizing the Input
    X = tf.keras.utils.normalize(X, axis=2)
    
    # Write Normalized Input to file
    write_to_file_input("data/FirstLayer/inputNorm.txt", X)
    
    
    # Write combination input to file
    X[:, :, 0] = 1
    X[:, :, 1] = 10
    X[:, :, 2] = 100
    write_to_file_input("data/FirstLayer/inputComb.txt", X)
    
    # Write all 1's input to file
    X[:, :, 0] = 1
    X[:, :, 1] = 1
    X[:, :, 2] = 1
    write_to_file_input("data/FirstLayer/inputSet1.txt", X)

    print("Creating Input files Complete")


In [ ]:
create_Input_Files()

In [ ]:
def write_to_file_weights_layer_one(fileName, W):
    fTemp = open(fileName, "w")
    for i in range(32):
        for j in range(3):
            for k in range(3):
                for l in range(3):
                    fTemp.write(str(W[k][l][j][i]) + "\n")
    fTemp.close()
    

In [ ]:
W = model.layers[2].get_weights()[0]
write_to_file_weights_layer_one("data/FirstLayer/weightsNorm.txt", W)

W[:,:,:,:] = 1
write_to_file_weights_layer_one("data/FirstLayer/weightsSet1.txt", W)

In [ ]:
def write_to_file_weights_layer_second(fileName, W):
    fTemp = open(fileName, "w")
    for i in range(1):
        for j in range(32):
            for k in range(3):
                for l in range(3):
                    fTemp.write(str(W[k][l][j][i]) + "\n")
    fTemp.close()


In [ ]:
W = model.layers[5].get_weights()[0]
write_to_file_weights_layer_second("data/SecondLayer/weightsNorm.txt", W)

W[:,:,:,:] = 1
write_to_file_weights_layer_second("data/SecondLayer/weightsSet1.txt", W)

In [3]:
def checkCalculations():

    img_width = 224
    img_height = 224

    img = image.load_img(fileName, target_size=(img_width, img_height))
    X = image.img_to_array(img)

    # Normalized the Input
    X = tf.keras.utils.normalize(X, axis=2)
    
    # Set as 1
    #X[:,:,0] = 1
    #X[:,:,1] = 1
    #X[:,:,2] = 1

    # Set as Combination
    #X[:,:,0] = 1
    #X[:,:,1] = 10
    #X[:,:,2] = 100

    print(X[0][0][0])
    print(X[0][1][0])
    print(X[0][2][0])
    X = np.expand_dims(X, axis=0)
    print(X.shape)

    # Weights part
    W = model.layers[2].get_weights()[0]
    # Weights set as 1
    #W[:,:,:,:] = 1
    W = np.expand_dims(W, axis=0)
    model.layers[2].set_weights(W)
    
    
    W = model.layers[5].get_weights()[0]
    # Weights set as 1
    #W[:,:,:,:] = 1
    W = np.expand_dims(W, axis=0)
    model.layers[5].set_weights(W)
    
    
    output = model.predict(X)
    print("Output Shape ---> ")
    print(output.shape)
    print("--------------------------------")

    # Save Output to file
    fOutput = open("data/SecondLayer/Second_Layer_Output.txt", "w")
    for i in range(len(output)):
        for j in range(len(output[0][0][0])):
            for k in range(len(output[0])):
                for l in range(len(output[0][0])):
                    fOutput.write(str(output[i][k][l][j]) + "\n")
    fOutput.close()
    print("Output File Writing Complete!!!")

    print("\n\n Description of Layers")
    for layer in model.layers:
        print(layer)


In [4]:
import time
start = time.time()
checkCalculations()
end = time.time()
print("Total Time -->", (end - start))


/home/nikhil/anaconda3/lib/python3.6/site-packages/numpy/linalg/linalg.py:2198: RuntimeWarning: invalid value encountered in sqrt
  return sqrt(add.reduce(s, axis=axis, keepdims=keepdims))


0.145017
0.351735
0.563547
(1, 224, 224, 3)
Output Shape ---> 
(1, 112, 112, 32)
--------------------------------
Output File Writing Complete!!!


 Description of Layers
Total Time --> 2.3383994102478027


<h2> Checking calculations for Layer 1 </h2>

In [ ]:
def checkFile_FirstLayer():
    fExp = open('data/FirstLayer/First_Layer_Output.txt', 'r')
    fAct = open('data/FirstLayer/output.txt', 'r')
    
    cExp = fExp.readlines()
    cAct = fAct.readlines()
    counter = 0
    for i in range(len(cExp)):
         if int(round(float(cAct[i].strip()))) != int(round(float(cExp[i].strip()))):
            counter += 1
        
    print("Number of mismatch - " + str(counter))
    
    fExp.close()
    fAct.close()
    
        

In [ ]:
checkFile_FirstLayer()

<h2> Checking calculations for Layer 2 </h2>

In [5]:
def checkFile_SecondLayer():
    fExp = open('data/SecondLayer/Second_Layer_Output.txt', 'r')
    fAct = open('data/SecondLayer/output.txt', 'r')
    
    cExp = fExp.readlines()
    cAct = fAct.readlines()
    counter = 0
    for i in range(len(cExp)):
         if int(round(float(cAct[i].strip()))) != int(round(float(cExp[i].strip()))):
            counter += 1
            print(str(cAct[i].strip()) + "--->" + str(cExp[i].strip()) )
        
    print("Number of mismatch - " + str(counter))
    
    fExp.close()
    fAct.close()
    
        

In [6]:
checkFile_SecondLayer()

9.401144--->6.0
8.770218--->6.0
7.427505--->6.0
8.773389--->6.0
8.625020--->6.0
8.615192--->6.0
8.518410--->6.0
8.309325--->6.0
8.343694--->6.0
8.288613--->6.0
8.289684--->6.0
8.193538--->6.0
8.323007--->6.0
8.213220--->6.0
8.469727--->6.0
8.020843--->6.0
8.074359--->6.0
8.095834--->6.0
8.041462--->6.0
8.358013--->6.0
8.377829--->6.0
8.375563--->6.0
8.428942--->6.0
8.246132--->6.0
8.337704--->6.0
8.183426--->6.0
8.117535--->6.0
8.161117--->6.0
8.192786--->6.0
8.196658--->6.0
8.203181--->6.0
8.103956--->6.0
8.109990--->6.0
8.180516--->6.0
8.188916--->6.0
8.228169--->6.0
8.059434--->6.0
8.419471--->6.0
7.747323--->6.0
8.224550--->6.0
8.215412--->6.0
8.168356--->6.0
8.152902--->6.0
8.219895--->6.0
8.216620--->6.0
8.215877--->6.0
8.232672--->6.0
8.262994--->6.0
7.957933--->6.0
8.411894--->6.0
7.258925--->6.0
8.912012--->6.0
7.521002--->6.0
6.562518--->6.0
7.157124--->6.0
9.709137--->6.0
6.901979--->6.0
8.790566--->6.0
8.768126--->6.0
7.039257--->6.0
7.015150--->6.0
8.452885--->6.0
9.597220

<h2> Saving Batch Norm for Layer 2 </h2>

In [ ]:
import math as m
batchNorm = model.layers[6].get_weights()

fMean = open("data/SecondLayer/Second_Layer_Mean.txt", "w")
fSD = open("data/SecondLayer/Second_Layer_StanDev.txt", "w")
fGamma = open("data/SecondLayer/Second_Layer_Gamma.txt", "w")
fBeta = open("data/SecondLayer/Second_Layer_Beta.txt", "w")


for i in range(len(batchNorm[0])):
    fGamma.write(str(float(batchNorm[0][i])) + "\n")

for i in range(len(batchNorm[1])):
    fBeta.write(str(float(batchNorm[1][i])) + "\n")

for i in range(len(batchNorm[2])):
    fMean.write(str(float(batchNorm[2][i])) + "\n")

for i in range(len(batchNorm[3])):
    # The Value for Epsilon is assumed to be 0.001 in this implementation
    fSD.write(str(m.sqrt(float(batchNorm[3][i]) + 0.001)) + "\n")


print(batchNorm[0])
print(batchNorm[1])
print(batchNorm[2])
print(batchNorm[3])

fMean.close()
fSD.close()
fGamma.close()
fBeta.close()


<h2> Layer Descriptions </h2>

In [2]:
for layer in model.layers:
    print(layer)
    
layer6 = model.layers[6]
W = np.array(layer6.get_weights())
print(W.shape)

(4, 32)
